## Remapping population density 
The aim of this little script is to remap all of the population density (from 1850 to 2014) and condense down into one file

In [43]:
%load_ext autoreload
%autoreload

import sys
sys.path.append('../')

import warnings
warnings.filterwarnings('ignore')

import os
from   io     import StringIO
import numpy  as np
import pandas as pd
import csv

import iris
import iris.plot as iplt
from iris.experimental.equalise_cubes import equalise_attributes
import matplotlib.pyplot as plt
import numpy.ma as ma
%matplotlib inline
import cartopy.crs as ccrs
from   libs.plot_maps    import *
from netCDF4 import Dataset

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [17]:
dir = '../data/pop_dens/'

year = range(1850,2015)
files = []

for y in year:
    files.append('PD_HYDEv3.2_' + str(y) + '.nc')

### Load in data
Merge all .nc files together 

In [20]:
aList = []

# Create a cube list
for f in files:
    dat = iris.load_cube(dir + f)
    aList.append(dat)

In [49]:
# Merge cube list together 
# cube_list = iris.cube.CubeList(aList)
# cubes = cube_list.merge()
# cubes = cube_list.concatenate_cube()
test = cubes[0:2]
equalise_attributes(test)
test[0].attributes()

TypeError: 'LimitedAttributeDict' object is not callable

Load the mask

In [ ]:
mask_dir = '..data/n96_masked_data/'
file = 'no_t_n96_mask.nc'

mask = iris.load_cube(mask_dir + file)

Plot data to check if it's loaded in correctly

In [ ]:
dat = cubes.collapsed('time', iris.analysis.MEAN)
dat.long_name = 'Population Density'
plot_lonely_cube(dat, cmap = 'magma', levels = None)

## Regrid
Using the mask n96e from 'regrid_nc' notebook, we're going to regrid the data. The main difference is the labelling of the lat and lon - both the original and the masked grid have the same number of coordinates

In [ ]:
m_cube = cubes.regrid(mask, iris.analysis.Linear())